In [64]:
import json
import spacy
import re
import difflib as dl
file_path = '/content/drive/MyDrive/NLP/gg2013.json'

#source1=https://catriscode.com/2021/03/02/extracting-or-removing-mentions-and-hashtags-in-tweets-using-python/
#source2== https://stackoverflow.com/questions/38365389/compare-similarity-between-names/38365913
#removed source2, didnt seem necessary

In [65]:
#this seemed unncessary but not deleting it yet. this taken from source2
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
    # print vec1, vec2
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    return Counter(WORD.findall(text))

def get_similarity(a, b):
    a = text_to_vector(a.strip().lower())
    b = text_to_vector(b.strip().lower())

    return get_cosine(a, b)

In [66]:
f = open(file_path)
data = json.load(f)
nlp = spacy.load("en_core_web_sm")
tweet_texts = []

for tweet in data:
  cleaned_tweet = re.sub('#([a-zA-Z0-9_]{1,50})', '', tweet['text'])
  cleaned_tweet = re.sub('@([a-zA-Z0-9_]{1,50})', '', cleaned_tweet)
  if 'rt @' not in cleaned_tweet:
    tweet_texts.append(cleaned_tweet)

In [67]:
host_key_words ='will be hosting|are hosting|is hosting|hosted'
hosts = {}
for tweet_text in tweet_texts:
  index = re.search(host_key_words,tweet_text)
  if (index != None):
    #print(tweet_text)
    doc = nlp(tweet_text[:index.start()])
    for entity in doc.ents:
      if entity.label_ == 'PERSON':
        if entity.text in hosts.keys():
          hosts[entity.text] = hosts[entity.text] + 1
        else:
          hosts[entity.text] = 1

In [68]:
hosts = dict(sorted(hosts.items(),key = lambda host:host[1],reverse=True))
print(hosts)

{'Amy Poehler': 94, 'Tina Fey': 81, 'Amy': 13, 'Tina': 9, 'Faye': 7, 'Amy Pohler': 4, 'Will Ferrell': 4, 'Jennifer Lawrence': 3, 'amy': 2, 'Anne Hathaway': 2, 'Kristen Wiig': 2, 'Kristin Wiig': 2, 'Anne': 2, 'Oscar': 2, 'Ricky Gerva': 2, 'Seth Macfarlane': 2, 'tina fey': 1, 'amy pohler': 1, "Giuliana Rancic's": 1, 'Norberry': 1, 'Any Poehler': 1, 'amy poehler': 1, 'tina': 1, 'Kate': 1, 'Angie': 1, 'mah girls tina': 1, 'Maggie Smith': 1, 'Liz': 1, 'Ricky': 1, 'Poehler': 1, 'Will': 1, 'Kristen': 1, 'Yay': 1, 'Neil Patrick Harr': 1, 'Sly': 1, 'Arnold': 1, 'LOL RT': 1, 'Les Miserables': 1, 'Ricky Gervais': 1, 'George Clooney': 1, 'RT  Great': 1, 'Golden Globes': 1}


In [69]:
actual_hosts = []
max = 0
for host in hosts.keys():
  if hosts[host] >=max:
    max = hosts[host]

for host in hosts.keys():
  if hosts[host]/max >=0.5:
    actual_hosts.append(host)
    #print(host+':'+str(hosts[host]/max))

print(actual_hosts)

['Amy Poehler', 'Tina Fey']
